# 1. Importación de paquetes

In [17]:
#Import packages
from modules import preprocess
from modules import training
import pandas as pd

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

## Paso 1: Lectura del *dataset* y obtención de datos

In [18]:
df = preprocess.load_csv_data('Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [19]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 2: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [20]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [21]:
#Process "Room_Occupancy_Count"
preprocess.recode_dataset_output(df)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

## Paso 3: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.

In [22]:
# Filter Date and Time columns
preprocess.remove_time_columns(df)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 4: División del *dataset* en *train* y *test*

In [23]:
# Split dataset into train and test
train_df, test_df = preprocess.split_dataset(df)
train_df.shape, test_df.shape

((7596, 17), (2533, 17))

In [24]:
train_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
6897,25.44,25.38,25.00,25.88,0,0,0,0,0.07,0.05,0.06,0.10,355,0.000000,0,0,0
4334,25.44,25.44,24.94,25.69,0,0,0,0,0.07,0.04,0.06,0.06,370,-0.100000,0,0,0
9955,25.06,25.06,24.56,25.13,1,1,10,5,0.08,0.04,0.06,0.10,345,0.000000,0,0,0
3942,25.69,25.69,25.31,25.88,0,0,0,0,0.07,0.05,0.06,0.06,650,-2.280769,0,0,0
1101,25.81,25.88,25.44,26.06,0,0,0,0,0.07,0.05,0.06,0.06,760,-2.042308,0,0,0


In [25]:
test_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
6390,25.44,25.44,25.19,26.00,0,0,0,0,0.08,0.05,0.06,0.09,355,-0.019231,0,0,0
2473,25.06,25.00,24.56,25.31,6,6,33,22,0.08,0.05,0.06,0.06,360,0.000000,0,0,0
5317,25.06,25.00,24.56,25.31,7,7,36,24,0.08,0.05,0.05,0.10,355,-0.238462,0,0,0
5758,25.31,25.31,25.19,25.94,17,20,74,53,0.07,0.05,0.07,0.10,355,0.000000,0,0,0
2882,25.50,25.31,25.06,25.88,122,37,67,49,0.47,0.24,0.10,0.09,370,0.280769,0,0,1


## Paso 5: División del *dataset* de entrenamiento en *features* y *output*

In [26]:
# Get features columns
X = preprocess.get_features(train_df)

# Get output column
y = preprocess.get_output(train_df)

In [27]:
X.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
6897,25.44,25.38,25.00,25.88,0,0,0,0,0.07,0.05,0.06,0.10,355,0.000000,0,0
4334,25.44,25.44,24.94,25.69,0,0,0,0,0.07,0.04,0.06,0.06,370,-0.100000,0,0
9955,25.06,25.06,24.56,25.13,1,1,10,5,0.08,0.04,0.06,0.10,345,0.000000,0,0
3942,25.69,25.69,25.31,25.88,0,0,0,0,0.07,0.05,0.06,0.06,650,-2.280769,0,0
1101,25.81,25.88,25.44,26.06,0,0,0,0,0.07,0.05,0.06,0.06,760,-2.042308,0,0


In [28]:
y.head()

6897    0
4334    0
9955    0
3942    0
1101    0
Name: Room_Occupancy_Count, dtype: int64

# 2. Entrenamiento y seguimiento de la huella de carbono con **CodeCarbon**

In [1]:
training.train_LR_eco2ai(df,0.25)

/home/misanchz/miniconda3/envs/TFG-Project/lib/python3.10/site-packages/eco2ai/emission_track.py:211: UserWarning: 
    If you use a VPN, you may have problems with identifying your country by IP.
    It is recommended to disable VPN or
    manually install the ISO-Alpha-2 code of your country during initialization of the Tracker() class.
    You can find the ISO-Alpha-2 code of your country here: https://www.iban.com/country-codes
    
  warnings.warn(
/home/misanchz/miniconda3/envs/TFG-Project/lib/python3.10/site-packages/eco2ai/tools/tools_gpu.py:35: NoGPUWarning: 

There is no any available GPU devices or your gpu is not supported by Nvidia library!
The thacker will consider CPU usage only

  warnings.warn(message="\n\nThere is no any available GPU devices or your gpu is not supported by Nvidia library!\nThe thacker will consider CPU usage only\n",


NameError: name 'X' is not defined

In [ ]:
df_emissions = pd.read_csv("eco2ai_emissions.csv",sep=",")

In [31]:
# Random Forest
#training.train_RF_codecarbon(X,y)

In [32]:
# Linear SVC (Support Vector Machines)
#training.train_SVC_codecarbon(X,y)